# Lab 02: Introduction to Text Preprocessing & the Spacy Toolkit

### Objectives:
1. Get familiar with basic text preprocessing pipelines
2. Get familiar with regular expressions, and the `re` package in Python
3. Evaluate the lexical diversity of the data in each category within the 20 News Groups Dataset
4. Use normalized BOW features to evaluate text similarity using the KL-divergence

### Required Reading:

1. https://universaldependencies.org/u/pos/
2. https://spacy.io/api/annotation#pos-tagging
3. https://spacy.io/api/annotation#dependency-parsing

# Part I: Introduction to Spacy

### Download Spacy's base English language *pipeline* components

``$ python -m spacy download en_core_web_sm``

What is a Spacy *pipeline*? A Spacy pipeline is an extensible tool that streamlines many of the common tasks in NLP, such as tokenization, part-of-speech tagging, named entity recognition, stemming, lemmatizing, and parsing. It also has custom pipeline components specifically for transformers. It is built for production use; much thought and care has gone into its API and implementation. You can actually configure Spacy to use some of the statistical models that we will discover in this class; for now we're just going to cover some of the basics.

In [1]:
import spacy

pipeline = spacy.load('en_core_web_sm')

### Download the 20 News Groups dataset using the sklearn package

This data consists of news articles from 20 different categories. 

In [2]:
from sklearn.datasets import fetch_20newsgroups
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

ng_train = fetch_20newsgroups(subset='train')
ng_test = fetch_20newsgroups(subset='test')
ng_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Get the number of training & test examples

In [3]:
len(ng_train.data), len(ng_test.data)

(11314, 7532)

### Take a peek at the first document and its label

In [4]:
ng_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [5]:
label_idx = ng_train.target[0]
ng_train.target_names[label_idx]

'rec.autos'

### Evaluate Spacy's recognition of entities, POS

In [7]:
from pprint import pprint

doc = pipeline(ng_train.data[0])

for i, token in enumerate(doc):
    pprint({"text": token.text,
            "lemma": token.lemma_,
            "POS": token.pos_,
            "tag": token.tag_,
            "dep": token.dep_,
            "shape": token.shape_,
            "is_alpha": token.is_alpha,
            "is_stop": token.is_stop})
    if i == 3:
        break

{'POS': 'ADP',
 'dep': 'prep',
 'is_alpha': True,
 'is_stop': True,
 'lemma': 'from',
 'shape': 'Xxxx',
 'tag': 'IN',
 'text': 'From'}
{'POS': 'PUNCT',
 'dep': 'punct',
 'is_alpha': False,
 'is_stop': False,
 'lemma': ':',
 'shape': ':',
 'tag': ':',
 'text': ':'}
{'POS': 'PROPN',
 'dep': 'pobj',
 'is_alpha': False,
 'is_stop': False,
 'lemma': 'lerxst@wam.umd.edu',
 'shape': 'xxxx@xxx.xxx.xxx',
 'tag': 'NNP',
 'text': 'lerxst@wam.umd.edu'}
{'POS': 'PUNCT',
 'dep': 'punct',
 'is_alpha': False,
 'is_stop': False,
 'lemma': '(',
 'shape': '(',
 'tag': '-LRB-',
 'text': '('}


### Visualize Spacy's dependency parse

In [8]:
from spacy import displacy

displacy.render(doc, style='dep')

### Let's define a preprocessing function that cleans our data

You'll notice that even the lemmatized text contains meaningless tokens. In the real world you're never going to get around having to do some feature engineering. In NLP this often means writing some regexes to transform text into a usable format. This has become less important in the deep learning era, but applying domain specific knowledge is always beneficial. In the case of this dataset, we have text that originated in news feeds, some of which is messy. There are email and url addresses, grammatical errors, and a lot puntuation and uninformative characters (e.g., the newline character `\n`). Below is a function that does some very basic regex (regular expression) matching to strip out emails, urls, punctuation, and other junk.

In [9]:
import re
from spacy.language import Language


# http://emailregex.com/
email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

# replace = [ (pattern-to-replace, replacement),  ...]
replace = [
    (r"<a[^>]*>(.*?)</a>", r"\1"),  # Matches most URLs
    (email_re, "email"),            # Matches emails
    (r"(?<=\d),(?=\d)", ""),        # Remove commas in numbers
    (r"\d+", "numbr"),              # Map digits to special token <numbr>
    (r"[\t\n\r\*\.\@\,\-\/]", " "),   # Punctuation and other junk
    (r"\s+", " ")                   # Stips extra whitespace
]

train_text = ng_train.data
test_text = ng_test.data
for repl in replace:
    train_text = [re.sub(repl[0], repl[1], text) for text in train_text]
    test_text = [re.sub(repl[0], repl[1], text) for text in test_text]

@Language.component("ng20")
def ng20_preprocess(doc):
    tokens = [token for token in doc 
              if not any((token.is_stop, token.is_punct))]
    tokens = [token.lemma_.lower().strip() for token in tokens]
    tokens = [token for token in tokens if token]
    return " ".join(tokens)

pipeline.add_pipe("ng20");

#### Peek at our processing pipeline

In [10]:
pipeline.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   tok2vec           doc.tensor                                          False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'ng20': {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': False}},
 'problems': {'tok2vec': [],
  'tagger': [],
 

### Now pass each training and test document through the pipeline

In [11]:
docs_train = [pipeline(doc) for doc in train_text[:500]]
docs_test = [pipeline(doc) for doc in test_text[:500]]

### Let's look at that first document following this transformation and compare it to the original text

In [12]:
ng_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [13]:
docs_train[0]

'email thing subject car nntp posting host racnumbr wam umd edu organization university maryland college park lines numbr wonder enlighten car see day numbr door sport car look late numbrs early numbrs call bricklin door small addition bumper separate rest body know tellme model engine spec year production car history info funky look car e mail thanks il bring neighborhood lerxst'

# Part II: Lexical diversity

Sometimes it's useful to understand how diverse is the language in some body of text. Once simple heuristic to evaluate diversity is as follows: 

$$ lexical\_diversity = \frac{ len(set(all\_words\_in\_doc)) }{ len(doc) }$$

Find the set of all words observed in the document, and divide it by the number of total words in the document. Let's use this to evalute the diversity of each category in the 20NG dataset.

### (5 pts) Task I: 
In the cell below, compute the diversity of each category in the 20NG dataset using the above heuristic

In [14]:
import re
import numpy as np
div_train = np.zeros(len(ng_train.target_names))

for i in range(len(div_train)):
    temp_set = set([])
    temp_total_words = 0
    for j in range(len(docs_train)):
        if(ng_train.target[j] == i):
            temp_doc = docs_train[j]
            temp_list = re.findall('[a-zA-Z]+', temp_doc)
            temp_total_words += len(temp_list)
            temp_set.update(set(temp_list))
    div_train[i] = len(temp_set) / temp_total_words

In [15]:
div_test = np.zeros(len(ng_test.target_names))

for i in range(len(div_test)):
    temp_set = set([])
    temp_total_words = 0
    for j in range(len(docs_test)):
        if(ng_test.target[j] == i):
            temp_doc = docs_test[j]
            temp_list = re.findall('[a-zA-Z]+', temp_doc)
            temp_total_words += len(temp_list)
            temp_set.update(set(temp_list))
    div_test[i] = len(temp_set) / temp_total_words

In [16]:
div_train

array([0.34491315, 0.42729306, 0.18966891, 0.42961418, 0.39568577,
       0.42356164, 0.43013699, 0.41701389, 0.45155393, 0.30111248,
       0.41614161, 0.43022821, 0.42914708, 0.43144273, 0.34041806,
       0.35241892, 0.42149035, 0.3579567 , 0.50166113, 0.51517572])

In [17]:
div_test

array([0.44570617, 0.44239812, 0.32947977, 0.3671151 , 0.40252591,
       0.4040849 , 0.39920027, 0.43581284, 0.43581284, 0.39857098,
       0.32543363, 0.44145247, 0.45016691, 0.46704583, 0.37913424,
       0.37698695, 0.45440301, 0.45478548, 0.36864175, 0.53290735])

In [18]:
np.argsort(div_train)

array([ 2,  9, 14,  0, 15, 17,  4, 10,  7, 16,  5,  1, 12,  3,  6, 11, 13,
        8, 18, 19], dtype=int64)

In [19]:
np.argsort(div_test)

array([10,  2,  3, 18, 15, 14,  9,  6,  4,  5,  7,  8, 11,  1,  0, 12, 16,
       17, 13, 19], dtype=int64)

In [20]:
ng_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Explain these scores: 

1. Is this result real or an artifact of some underlying problem with our data? 
2. What might you do to better evaluate lexical diversity on this data using this scoring function?
3. Is this heuristic a good metric for lexical diversity in general?

### Explaination:
1. We only use 500 examples for training set and testing set each.(Time restricted by the pipeline)
2. I think one option is to remove common words.
3. I think this heuristic is not a good metric in general because there are so many topics for newpapers. We need to use a more stable value to identify the topic in general. And we see in training and testing dataset the diversity of the same topic changes somehow.

### Entropy 
Entropy is another, perhaps more principled, way by which we can evaluate how diverse, or varied, is a piece of text. Recall the definition of Entropy, $H(P(x))$:

$$ H(P(x)) = \sum_{i=1}^{N} -P(x_{i}) \log P(x_{i}) $$

In the Bag-of-Words (BOW) feature representation of a document, each document is represented by a word count vector, ${x}_{i} \in \mathbb{R}^{N}$ where $N$ is the cardinality of the set of words in the document.

### (10 pts) Task II:
In order to compute an entropy from this representation, you'll first need to convert those count vectors into probability distributions. Then compute the entropy of the word distributions for each news category.

In [21]:
from collections import Counter
ent_train = np.zeros(len(ng_train.target_names))

for i in range(len(div_train)):
    temp_count = Counter([])
    for j in range(len(docs_train)):
        if(ng_train.target[j] == i):
            temp_doc = docs_train[j]
            temp_list = re.findall('[a-zA-Z]+', temp_doc)
            temp_count = temp_count + Counter(temp_list)
    total_words = np.array([v for k, v in temp_count.items()]).sum()
    ent_train[i] = np.array([-v/total_words*np.log(v/total_words) for k, v in temp_count.items()]).sum()

In [22]:
ent_test = np.zeros(len(ng_test.target_names))

for i in range(len(div_test)):
    temp_count = Counter([])
    for j in range(len(docs_test)):
        if(ng_test.target[j] == i):
            temp_doc = docs_test[j]
            temp_list = re.findall('[a-zA-Z]+', temp_doc)
            temp_count = temp_count + Counter(temp_list)
    total_words = np.array([v for k, v in temp_count.items()]).sum()
    ent_test[i] = np.array([-v/total_words*np.log(v/total_words) for k, v in temp_count.items()]).sum()

In [23]:
ent_train

array([6.88802052, 6.41874406, 5.70124812, 5.95246069, 6.35550718,
       6.04702797, 5.97392409, 6.32284914, 6.65583839, 4.76594137,
       6.53696846, 6.46189682, 6.46826275, 6.77541367, 6.7076685 ,
       6.8688232 , 6.56570795, 6.76263537, 6.65201333, 6.03562368])

In [24]:
ent_test

array([6.64074489, 6.44602536, 6.14365639, 6.31009098, 6.39548117,
       6.18077632, 6.01643421, 6.57543523, 6.62742945, 6.35284799,
       5.91099278, 6.4062538 , 6.21354488, 6.73665974, 6.65666899,
       6.70941329, 6.77880095, 6.69446953, 7.03577108, 6.31605756])

In [25]:
np.argsort(ent_train)

array([ 9,  2,  3,  6, 19,  5,  7,  4,  1, 11, 12, 10, 16, 18,  8, 14, 17,
       13, 15,  0], dtype=int64)

In [26]:
np.argsort(ent_test)

array([10,  6,  2,  5, 12,  3, 19,  9,  4, 11,  1,  7,  8,  0, 14, 17, 15,
       13, 16, 18], dtype=int64)

In [27]:
ng_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Explain this result

1. What does it mean for a distribution to have high or low entropy?
2. Do these scores make intuitive sense? Any more or less so than the heuristic from Task I?
2. Is entropy a good metric for evaluating lexical diversity in general?

### Explaination:
1. High entropy: word frequencies are distributed differently. Low entropy: word frequencies are distributed similarly.
2. Yes. More heuristic.
3. I think entropy is a good metric for evaluating lexical diversity in general because for the same topic the range doesn't change a lot. It's more stable.

# Part III: Document Similarity

Throughout this course we will discuss the notion of *similarity* between texts and explore ways to measure it. This is a critical component of search and recommender systems. One such approach involves measuring how *close* two word distributions are using the notion divergence, which we discussed in the first lecture.

### (10 pts) Task III

Using the definition below, compute the KL-divergence, $K_{DL}$, between the word distributions in each category. This will result in a $K \times K$ matrix of divergence values.

$$ D_{KL}(P||Q) = \sum_{i=1}^{N} P(x_{i}) \log \frac{P(x_{i})}{Q(x_{i})} $$

In [34]:
dis_res = []
kldiv_train = np.zeros((len(docs_train),len(docs_train)))


for j in range(len(docs_train)):
    temp_doc = docs_train[j]
    temp_list = re.findall('[a-zA-Z]+', temp_doc)
    temp_count = Counter(temp_list)
    temp_total = np.array([v for k, v in temp_count.items()]).sum()
    temp_count_new = Counter({k:(v/temp_total) for k, v in temp_count.items()})
    dis_res.append(temp_count_new)
    

In [35]:
for i in range(len(dis_res)):
    for j in range(len(dis_res)):
        ss = 0
        if i != j:
            count1 = dis_res[i]
            count2 = dis_res[j]
            count_union = count1.keys() & count2.keys()
            for v in count_union:
                p = count1[v]
                q = count2[v]
                ss = ss + p*np.log(p/q)
            kldiv_train[i][j] = ss

In [36]:
ng_train.target[:len(docs_train)]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4,  8, 19,  4, 14,  6,  0,  1,
        7, 12,  5,  0, 10,  6,  2,  4,  1, 12,  9, 15,  7,  6, 13, 12, 17,
       18, 10,  8, 11,  8, 16,  9,  4,  3,  9,  9,  4,  4,  8, 12, 14,  5,
       15,  2, 13, 17, 11,  7, 10,  2, 14, 12,  5,  4,  6,  7,  0, 11, 16,
        0,  6, 17,  7, 12,  7,  3, 12, 11,  7,  2,  2,  0, 16,  1,  2,  7,
        3,  2,  1, 10, 12, 12, 17, 12,  2,  8,  8, 18,  5,  0,  1,  6, 12,
        8,  4, 17, 12, 12, 12,  1,  6, 18,  4,  3, 10,  9,  0, 13, 11,  5,
       14, 15,  8,  4, 15, 15,  1,  0, 16,  9,  8,  6, 13,  6, 17, 14,  0,
        9,  1,  2, 15, 13,  9,  2,  8,  2, 13,  2,  0, 15, 14,  1, 14, 17,
       14,  4,  4,  7, 19,  1, 15, 17, 16,  2, 15,  9, 12,  6,  9,  6,  6,
       18,  1, 10,  6, 10,  5,  2, 13,  3,  9, 13, 12, 13,  8,  4,  3,  9,
        1, 12,  4,  2,  2, 11, 13,  4,  1, 12,  0, 16, 12, 16,  7, 17, 15,
       11, 14,  2,  7, 10, 14, 15,  5, 16, 11,  4, 13,  7,  4, 13, 17,  1,
       15, 17, 17,  9, 16

In [37]:
kldiv_train[:5,:5]

array([[ 0.        ,  0.0266388 ,  0.13126456,  0.05960672,  0.03311527],
       [-0.00361625,  0.        ,  0.09087954, -0.00353957,  0.02132274],
       [-0.02813985, -0.03586257,  0.        , -0.02619227, -0.00716   ],
       [-0.01166737,  0.01195987,  0.08291768,  0.        ,  0.02078936],
       [ 0.00739263,  0.00654307,  0.13179685,  0.02937914,  0.        ]])

In [293]:
kldiv_train[9,2]

-0.0468674767312531

### Explain this result

1. How did you handle any differences in the support for P and Q? What about when Q(x) = 0?
2. What does it mean for two distributions to have high or low divergence?
3. Do these similarity scores make sense intuitively?
4. Is the resultant $K \times K$ matrix symmetric? Why is this the case?
5. Is $D_{KL}$ a good measure of the similarity in this context?   

### Explaination:
1. When $P(x)=0$ and $Q(x)\neq0$ $$P(x)log\frac{P(x)}{Q(x)}=P(x)logP(x)-P(x)logQ(x)$$
Since $\lim_{P(x)\rightarrow0^+}P(x)logP(x)=0$ We have the above expression equal to 0.  
When $P(x)\neq0$ and $Q(x)=0$, since $\lim_{Q(x)\rightarrow0^{+}}P(x)log\frac{P(x)}{Q(x)}=0$
2. High divergence: P is different from Q. Low divergence: P is similar to Q.  
3. Sometimes not.
4. No. KL divergence is not a symmetric expression for P & Q.
5. I don't think it's a good measure because sometimes $D_{KL}(P||Q)$ and $D_{KL}(Q||P)$ give different opinions about the similarity.